TV RASPORED

In [ ]:
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
import glob
import xml.etree.ElementTree as ET
import os
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
from datetime import timedelta
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
#/content/raspored/HRT3-01-01-21-06_03_03.xml
filenames = glob.glob("/content/hrt_svi/*.xml")
ukupno = []
s = []
series = []
p = []
repriza = []
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
for filename in filenames:
  tree = ET.parse(filename)
  root = tree.getroot()
  i=0
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
  if (os.path.basename(filename)[15:16] == '6'):
    start_d = os.path.basename(filename)[5:13]
    stop_d = start_d
    start_t = '06:10'
    stop_t = '18:00'
    start = start_d + ' ' + start_t
    start = datetime.strptime(start, '%m-%d-%y %H:%M')
    stop = stop_d + ' ' + stop_t
    stop = datetime.strptime(stop, '%m-%d-%y %H:%M')
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
  else:
    start_d = os.path.basename(filename)[5:13]
    stop_d = start_d
    start_t = '18:00'
    stop_t = '06:10'
    start = start_d + ' ' + start_t
    start = datetime.strptime(start, '%m-%d-%y %H:%M')
    #stop_d = start_d + timedelta(days=1)
    stop = stop_d + ' ' + stop_t
    stop = datetime.strptime(stop, '%m-%d-%y %H:%M')
    #stop = stop + dt.timedelta(hours=12)
    stop = stop + dt.timedelta(days=1)
    #print('start')
    #print(start)
    #print('stop')
    #print(stop)
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
  for programme in root.iter('programme'):
    series = pd.Series(programme.attrib)
    s=series
    p.append(s)
    ch = s['channel']
    start_p = s['start']
    start_p = start_p[0:10]+start_p[12:18]
    start_pp = datetime.strptime(start_p, '%d.%m.%Y %H:%M')
#-----------------------------------------------------------------------------
    stop_p =s['stop']
    stop_p = stop_p[0:10]+stop_p[12:18]
    stop_pp = datetime.strptime(stop_p, '%d.%m.%Y %H:%M')
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
    title=''
    for title1 in (programme.iter('title')):
      #print(title1.text)
      title = title + title1.text
      #title = title1.text
      if "(R)" in title1.text:
        repriza='da'
      else:
        repriza='ne'
    
    cat=''
    for c in (programme.iter('category')):
      #cat = cat + c.text
      cat = c.text

    descr=''
    for ds in (programme.iter('desc')):
      #descr = descr + ds.text
      descr = ds.text
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------
    #i=0
    if (start_pp >= start and start_pp < stop):
      
      #for child in s:
      ukupno.append({
        'channel':ch,
        'start':start_p,
        'stop':stop_p,
        'title':title,
        'kategorija':cat,
        'descr':descr,
        'repriza':repriza,
        })
        #i=i+1
        #i=i+1
#-----------------------------------------------------------------------------
#-----------------------------------------------------------------------------


In [ ]:
df = pd.DataFrame(ukupno)

In [ ]:
df

,channel,start,stop,title,kategorija,descr,repriza
0,hrt - htv3.tv.hrt.hr,07.03.2021 18:15,07.03.2021 18:20,Hrvati koji su mijenjali svijet (1): Ivan Vuče...,ZNANOST,Oni su zadužili ne samo Hrvatsku nego i cijeli...,da
1,hrt - htv3.tv.hrt.hr,07.03.2021 18:20,07.03.2021 18:41,Dani Orisa 2020. - Bernard Khourty,UMJETNOST I KULTURA,U prosincu 2020. su u OKA-i održani dvadeseti ...,ne
2,hrt - htv3.tv.hrt.hr,07.03.2021 18:41,07.03.2021 19:50,Vrijeme je za jazz: Slavimo 800,GLAZBA,Jedina emisija Hrvatske radiotelevizije koja s...,ne
3,hrt - htv3.tv.hrt.hr,07.03.2021 19:50,07.03.2021 20:05,Fotografija u Hrvatskoj: Miljenko Horvat (R),UMJETNOST I KULTURA,U formi kratkog dokumentarnog filma emisija pr...,da
4,hrt - htv3.tv.hrt.hr,07.03.2021 20:05,07.03.2021 20:25,Šahovski komentar: Kurajica - Hübner,OBRAZOVANJE,Analiza najboljih šahovskih partija uz komenta...,ne
...,...,...,...,...,...,...,...
14100,hrt - htv4.tv.hrt.hr,21.11.2020 16:58,21.11.2020 17:00,Vrijeme Europa danas (R),VIJESTI,,da
14101,hrt - htv4.tv.hrt.hr,21.11.2020 17:00,21.11.2020 17:15,Vijesti u 17,VIJESTI,,ne
14102,hrt - htv4.tv.hrt.hr,21.11.2020 17:15,21.11.2020 17:40,Prometej (R),ZNANOST,,da
14103,hrt - htv4.tv.hrt.hr,21.11.2020 17:40,21.11.2020 17:50,Bajkovita Hrvatska: NP Plitvička jezera (R),DRUGI INFORMATIVNI SADRŽAJI,"Plitvička jezera - čarobna igra vode, zelenila...",da


In [ ]:
df['start'] = pd.to_datetime(df['start'], format='%d.%m.%Y %H:%M')
df.sort_values(by=['start'], inplace=True)
df.replace('hrt - htv3.tv.hrt.hr', 'HRT3 HD', inplace=True)
df.replace('hrt - htv4.tv.hrt.hr', 'HRT4 HD', inplace=True)
df = df.reset_index(drop=True)
df

,channel,start,stop,title,kategorija,descr,repriza
0,HRT3 HD,2020-11-12 06:20:00,12.11.2020 07:04,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da
1,HRT4 HD,2020-11-12 07:00:00,12.11.2020 07:10,Vijesti,VIJESTI,,ne
2,HRT3 HD,2020-11-12 07:04:00,12.11.2020 08:00,Avionima na kraj svijeta: Quebec - zračni omni...,OBRAZOVANJE,,da
3,HRT4 HD,2020-11-12 07:10:00,12.11.2020 07:15,Poslovne vijesti,VIJESTI,,ne
4,HRT4 HD,2020-11-12 07:15:00,12.11.2020 07:27,Vijesti iz kulture (R),VIJESTI,,da
...,...,...,...,...,...,...,...
14100,HRT4 HD,2021-03-24 16:59:00,24.03.2021 17:14,Vijesti u 17,VIJESTI,Najnovije vijesti iz zemlje i svijeta.,ne
14101,HRT4 HD,2021-03-24 17:14:00,24.03.2021 17:29,Županijska panorama Čakovec i Varaždin,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne
14102,HRT4 HD,2021-03-24 17:29:00,24.03.2021 17:44,Županijska panorama Šibenik,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne
14103,HRT4 HD,2021-03-24 17:44:00,24.03.2021 17:59,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne


In [ ]:
from datetime import datetime as dt
from datetime import timedelta
import numpy as np
import datetime
s = df['start'][0]
p = df['start'].iloc[-1]
raz = p - s
min = divmod(raz.total_seconds(), 60)
m = int(min[0])
base = pd.to_datetime(df['start'][0], format='%d.%m.%Y %H:%M')
#arr = np.array([base + timedelta(minutes=i) for i in range(len(p))])
arr = np.array([base + datetime.timedelta(minutes=i) for i in range(0, m)])
df_d = pd.DataFrame({'date': arr})

In [ ]:
df_d

,date
0,2020-11-12 06:20:00
1,2020-11-12 06:21:00
2,2020-11-12 06:22:00
3,2020-11-12 06:23:00
4,2020-11-12 06:24:00
...,...
190774,2021-03-24 17:54:00
190775,2021-03-24 17:55:00
190776,2021-03-24 17:56:00
190777,2021-03-24 17:57:00


In [ ]:
df['start'] = pd.to_datetime(df['start'], format='%d.%m.%Y %H:%M')
df['stop'] = pd.to_datetime(df['stop'], format='%d.%m.%Y %H:%M')

In [ ]:
df_merged = df.merge(df_d, how='right', left_on='start', right_on='date')

In [ ]:
df_merged.sort_values(by=['date'], inplace=True)

In [ ]:
df_merged['start'] = df_merged['start'].fillna(method='ffill')
df_merged['stop'] = df_merged['stop'].fillna(method='ffill')
df_merged['channel'] = df_merged['channel'].fillna(method='ffill')
df_merged['title'] = df_merged['title'].fillna(method='ffill')
df_merged['descr'] = df_merged['descr'].fillna(method='ffill')
df_merged['repriza'] = df_merged['repriza'].fillna(method='ffill')
df_merged['kategorija'] = df_merged['kategorija'].fillna(method='ffill')

In [ ]:
df_merged = df_merged[['channel','title','kategorija', 'descr', 'repriza', 'start', 'stop', 'date']]

In [ ]:
df_merged.reset_index(inplace=True, drop=True)

In [ ]:
df_merged['date'] = df_d['date']

In [ ]:
df_merged.sort_values(by=['date'], inplace=True)

In [ ]:
df_merged.head()

,channel,title,kategorija,descr,repriza,start,stop,date
0,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:20:00
1,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:21:00
2,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:22:00
3,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:23:00
4,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:24:00


In [ ]:
df_merged['date'] = pd.to_datetime(df_merged['date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_merged.sort_values(by=['start'], inplace=True)

In [ ]:
df_merged

,channel,title,kategorija,descr,repriza,start,stop,date
0,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:20:00
23,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:43:00
24,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:44:00
25,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:45:00
26,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:46:00
...,...,...,...,...,...,...,...,...
191635,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT
191634,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT
191647,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT
191640,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT


GLEDANOST

In [ ]:
import pandas as pd
import glob
# /content/_ukupno_SKRACENO.csv
df2 = pd.read_csv(r'/content/_ukupno_SKRACENO.csv')

In [ ]:
df2

,channel_name,datum,h,m,broj_gledatelja,vrijeme
0,HRT3 HD,11/13/2020,0,0,553,0:00:00
1,HRT4 HD,11/13/2020,0,0,397,0:00:00
2,HRT3 HD,11/13/2020,0,1,541,0:01:00
3,HRT4 HD,11/13/2020,0,1,405,0:01:00
4,HRT3 HD,11/13/2020,0,2,511,0:02:00
...,...,...,...,...,...,...
324637,HRT4 HD,3/25/2021,9,57,1141,9:57:00
324638,HRT3 HD,3/25/2021,9,58,193,9:58:00
324639,HRT4 HD,3/25/2021,9,58,1118,9:58:00
324640,HRT3 HD,3/25/2021,9,59,181,9:59:00


In [ ]:
df2['datum'] = pd.to_datetime(df2['datum'], format='%m/%d/%Y')

In [ ]:
df2['vrijeme'] = pd.to_datetime(df2['vrijeme'], format='%H:%M:%S').dt.time

In [ ]:
df2

,channel_name,datum,h,m,broj_gledatelja,vrijeme
0,HRT3 HD,2020-11-13,0,0,553,00:00:00
1,HRT4 HD,2020-11-13,0,0,397,00:00:00
2,HRT3 HD,2020-11-13,0,1,541,00:01:00
3,HRT4 HD,2020-11-13,0,1,405,00:01:00
4,HRT3 HD,2020-11-13,0,2,511,00:02:00
...,...,...,...,...,...,...
324637,HRT4 HD,2021-03-25,9,57,1141,09:57:00
324638,HRT3 HD,2021-03-25,9,58,193,09:58:00
324639,HRT4 HD,2021-03-25,9,58,1118,09:58:00
324640,HRT3 HD,2021-03-25,9,59,181,09:59:00


In [ ]:
df2[['datum_ukupno']] = df2['datum'].astype(str) + ' ' + df2['vrijeme'].astype(str)

SPAJANJE

In [ ]:
df_merged['date'] = pd.to_datetime(df_merged['date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_merged[23:]

,channel,title,kategorija,descr,repriza,start,stop,date
2,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:22:00
3,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:23:00
4,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:24:00
5,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:25:00
6,HRT3 HD,"Betty i njezine dijagnoze (1), serija (12) (5/...",IGRANI PROGRAM,,da,2020-11-12 06:20:00,2020-11-12 07:04:00,2020-11-12 06:26:00
...,...,...,...,...,...,...,...,...
191635,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT
191634,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT
191647,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT
191640,HRT4 HD,Županijska panorama Osijek,VIJESTI,Informativna emisija koja donosi najzanimljivi...,ne,2021-03-24 17:44:00,2021-03-24 17:59:00,NaT


In [ ]:
df2['datum_ukupno'] = pd.to_datetime(df2['datum_ukupno'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
dm = df_merged.merge(df2, how='inner', left_on=['channel','date'], right_on=['channel_name','datum_ukupno'])

In [ ]:
#dm.sort_values(by=['datum_ukupno'], inplace=True)

In [ ]:
dm.sort_values(by=['date'])

,channel,title,kategorija,descr,repriza,start,stop,date,channel_name,datum,h,m,broj_gledatelja,vrijeme,datum_ukupno
10,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:00:00,HRT4 HD,2020-11-13,0,0,397,00:00:00,2020-11-13 00:00:00
9,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:01:00,HRT4 HD,2020-11-13,0,1,405,00:01:00,2020-11-13 00:01:00
8,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:02:00,HRT4 HD,2020-11-13,0,2,395,00:02:00,2020-11-13 00:02:00
7,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:03:00,HRT4 HD,2020-11-13,0,3,368,00:03:00,2020-11-13 00:03:00
6,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:04:00,HRT4 HD,2020-11-13,0,4,356,00:04:00,2020-11-13 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160508,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:54:00,HRT4 HD,2021-03-24,17,54,1318,17:54:00,2021-03-24 17:54:00
160500,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:55:00,HRT4 HD,2021-03-24,17,55,1291,17:55:00,2021-03-24 17:55:00
160499,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:56:00,HRT4 HD,2021-03-24,17,56,1236,17:56:00,2021-03-24 17:56:00
160501,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:57:00,HRT4 HD,2021-03-24,17,57,1258,17:57:00,2021-03-24 17:57:00


In [ ]:
dm.head()

,channel,title,kategorija,descr,repriza,start,stop,date,channel_name,datum,h,m,broj_gledatelja,vrijeme,datum_ukupno
0,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:10:00,HRT4 HD,2020-11-13,0,10,313,00:10:00,2020-11-13 00:10:00
1,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:09:00,HRT4 HD,2020-11-13,0,9,341,00:09:00,2020-11-13 00:09:00
2,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:08:00,HRT4 HD,2020-11-13,0,8,358,00:08:00,2020-11-13 00:08:00
3,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:07:00,HRT4 HD,2020-11-13,0,7,375,00:07:00,2020-11-13 00:07:00
4,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:06:00,HRT4 HD,2020-11-13,0,6,362,00:06:00,2020-11-13 00:06:00


In [ ]:
dm.drop(['datum', 'h', 'm', 'channel_name'], axis=1, inplace=True)

In [ ]:
#-----------------DODANO----------------------------------------
dm.sort_values(by=['date'], inplace=True, ignore_index=True)
#-------------------------------------------------------

In [ ]:
dm.head()

,channel,title,kategorija,descr,repriza,start,stop,date,broj_gledatelja,vrijeme,datum_ukupno
0,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:00:00,397,00:00:00,2020-11-13 00:00:00
1,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:01:00,405,00:01:00,2020-11-13 00:01:00
2,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:02:00,395,00:02:00,2020-11-13 00:02:00
3,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:03:00,368,00:03:00,2020-11-13 00:03:00
4,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13,2020-11-13 00:04:00,356,00:04:00,2020-11-13 00:04:00


In [ ]:
dm.drop(columns=["datum_ukupno"])

,channel,title,kategorija,descr,repriza,start,stop,date,broj_gledatelja,vrijeme
0,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:00:00,397,00:00:00
1,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:01:00,405,00:01:00
2,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:02:00,395,00:02:00
3,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:03:00,368,00:03:00
4,HRT4 HD,Pogled preko granice - Hrvati u BiH (R),DRUGI INFORMATIVNI SADRŽAJI,,da,2020-11-12 23:35:00,2020-11-13 00:00:00,2020-11-13 00:04:00,356,00:04:00
...,...,...,...,...,...,...,...,...,...,...
160514,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:54:00,1318,17:54:00
160515,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:55:00,1291,17:55:00
160516,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:56:00,1236,17:56:00
160517,HRT4 HD,Susret u dijalogu: Štovanje sv. Josipa u hrvat...,RELIGIJA,Štovanje sv. Josipa u hrvatskom narodu tema je...,ne,2021-03-24 03:10:00,2021-03-24 04:00:00,2021-03-24 17:57:00,1258,17:57:00


In [ ]:
dm3 = dm.loc[dm['channel'] == 'HRT3 HD']
dm4 = dm.loc[dm['channel'] == 'HRT4 HD']

In [ ]:
dm3.to_csv(r'/content/\total3.csv', index=False)
dm4.to_csv(r'/content/\total4.csv', index=False)

PRIPREMA ZA UCENJE

In [ ]:
print(dm['kategorija'].unique())

['VIJESTI' 'ZABAVA' 'OBRAZOVANJE' 'UMJETNOST I KULTURA' 'ZNANOST'
 'DRUGI INFORMATIVNI SADRŽAJI' 'RELIGIJA' 'GLAZBA']


In [ ]:
kod_tv_kanal = {'HRT3 HD': 0,
                'HRT4 HD': 1}

In [ ]:
kod_kategorija = {'IGRANI PROGRAM': 0,
                  'DRUGI INFORMATIVNI SADRŽAJI': 1,
                  'GLAZBA': 2,
                  'UMJETNOST I KULTURA': 3,
                  'OBRAZOVANJE': 4,
                  'ZNANOST': 5,
                  'VIJESTI': 6,
                  'RELIGIJA': 7,
                  'ZABAVA': 8,
                  'PROMOCIJA': 9}

In [ ]:
kod_repriza = {'ne': 0,
               'da': 1}

In [ ]:
dm[['godina']] = dm['date'].dt.year
dm[['mjesec']] = dm['date'].dt.month
dm[['dan']] = dm['date'].dt.day
dm[['sat']] = dm['date'].dt.hour
dm[['minuta']] = dm['date'].dt.minute

In [ ]:
dm[['godina_start']] = dm['start'].dt.year
dm[['mjesec_start']] = dm['start'].dt.month
dm[['dan_start']] = dm['start'].dt.day
dm[['sat_start']] = dm['start'].dt.hour
dm[['minuta_start']] = dm['start'].dt.minute

In [ ]:
dm[['godina_stop']] = dm['stop'].dt.year
dm[['mjesec_stop']] = dm['stop'].dt.month
dm[['dan_stop']] = dm['stop'].dt.day
dm[['sat_stop']] = dm['stop'].dt.hour
dm[['minuta_stop']] = dm['stop'].dt.minute

In [ ]:
dm.sort_values(by=['date'], inplace=True)

In [ ]:
dm.drop(columns=['start', 'stop', 'date'], inplace=True)

In [ ]:
idx = np.unique(dm['title'], return_inverse=True)

In [ ]:
labels, levels = pd.factorize(dm.title)

In [ ]:
d = dict([(y, x) for x,y in enumerate(sorted(set(levels)))])

In [ ]:
d

{'"Ovčara - vječna rana" - polaganje vijenaca, prigodni program i molitva': 0,
 '"Žrtva Borovo naselja za Domovinu", prigodni program i polaganje vijenaca': 1,
 '#BUDI PROMJENA: Akcija Gastro kluba, dokumentarna reportaža\xa0(R)': 2,
 '#BUDI PROMJENA: Alpinisti pomažu, dokumentarna reportaža\xa0(R)': 3,
 '#BUDI PROMJENA: Dan planeta Zemlje, dokumentarna reportaža\xa0(R)': 4,
 '#BUDI PROMJENA: Dr. Alen, dokumentarna reportaža\xa0(R)': 5,
 '#BUDI PROMJENA: Gajnice za Otta, dokumentarna reportaža\xa0(R)': 6,
 '#BUDI PROMJENA: Inovator automobil, dokumentarna reportaža\xa0(R)': 7,
 '#BUDI PROMJENA: Inovatorica, dokumentarna reportaža\xa0(R)': 8,
 '#BUDI PROMJENA: Instrukcije tete Nikoline, dokumentarna reportaža\xa0(R)': 9,
 '#BUDI PROMJENA: Jedni za druge, dokumentarna reportaža\xa0(R)': 10,
 '#BUDI PROMJENA: Kristina Panić, dokumentarna reportaža\xa0(R)': 11,
 '#BUDI PROMJENA: Markuševčani pomažu, dokumentarna reportaža\xa0(R)': 12,
 '#BUDI PROMJENA: Medicinska sestra Romana, dokumentarn

In [ ]:
len(d)

1054

In [ ]:
tuple(d.items())[1023]

('Zdrav život', 1023)

In [ ]:
dm.replace({'channel': kod_tv_kanal,'title': d, 'kategorija': kod_kategorija, 'repriza': kod_repriza}, inplace=True)

In [ ]:
dm.head()

,channel,title,kategorija,repriza,broj_gledatelja,vrijeme,datum_ukupno,godina,mjesec,dan,sat,minuta,godina_start,mjesec_start,dan_start,sat_start,minuta_start,godina_stop,mjesec_stop,dan_stop,sat_stop,minuta_stop
8,1,988,6,0,397,00:00:00,2020-11-13 00:00:00,2020,11,13,0,0,2020,11,13,0,0,2020,11,13,0,10
7,1,988,6,0,405,00:01:00,2020-11-13 00:01:00,2020,11,13,0,1,2020,11,13,0,0,2020,11,13,0,10
5,1,988,6,0,395,00:02:00,2020-11-13 00:02:00,2020,11,13,0,2,2020,11,13,0,0,2020,11,13,0,10
6,1,988,6,0,368,00:03:00,2020-11-13 00:03:00,2020,11,13,0,3,2020,11,13,0,0,2020,11,13,0,10
9,1,988,6,0,356,00:04:00,2020-11-13 00:04:00,2020,11,13,0,4,2020,11,13,0,0,2020,11,13,0,10


In [ ]:
dm.drop(['godina_start', 'godina_stop', 'godina'],axis=1, inplace=True)

In [ ]:
dm.head()

,channel,title,kategorija,repriza,broj_gledatelja,vrijeme,datum_ukupno,mjesec,dan,sat,minuta,mjesec_start,dan_start,sat_start,minuta_start,mjesec_stop,dan_stop,sat_stop,minuta_stop
8,1,988,6,0,397,00:00:00,2020-11-13 00:00:00,11,13,0,0,11,13,0,0,11,13,0,10
7,1,988,6,0,405,00:01:00,2020-11-13 00:01:00,11,13,0,1,11,13,0,0,11,13,0,10
5,1,988,6,0,395,00:02:00,2020-11-13 00:02:00,11,13,0,2,11,13,0,0,11,13,0,10
6,1,988,6,0,368,00:03:00,2020-11-13 00:03:00,11,13,0,3,11,13,0,0,11,13,0,10
9,1,988,6,0,356,00:04:00,2020-11-13 00:04:00,11,13,0,4,11,13,0,0,11,13,0,10


In [ ]:
dm.sort_values(by=['datum_ukupno'])

,channel,title,kategorija,repriza,broj_gledatelja,vrijeme,datum_ukupno,mjesec,dan,sat,minuta,mjesec_start,dan_start,sat_start,minuta_start,mjesec_stop,dan_stop,sat_stop,minuta_stop
8,1,988,6,0,397,00:00:00,2020-11-13 00:00:00,11,13,0,0,11,13,0,0,11,13,0,10
7,1,988,6,0,405,00:01:00,2020-11-13 00:01:00,11,13,0,1,11,13,0,0,11,13,0,10
5,1,988,6,0,395,00:02:00,2020-11-13 00:02:00,11,13,0,2,11,13,0,0,11,13,0,10
6,1,988,6,0,368,00:03:00,2020-11-13 00:03:00,11,13,0,3,11,13,0,0,11,13,0,10
9,1,988,6,0,356,00:04:00,2020-11-13 00:04:00,11,13,0,4,11,13,0,0,11,13,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155467,1,146,1,1,269,06:03:00,2021-03-21 06:03:00,3,21,6,3,3,21,4,5,3,21,4,35
155468,1,146,1,1,256,06:04:00,2021-03-21 06:04:00,3,21,6,4,3,21,4,5,3,21,4,35
155469,1,146,1,1,255,06:05:00,2021-03-21 06:05:00,3,21,6,5,3,21,4,5,3,21,4,35
155470,1,146,1,1,246,06:06:00,2021-03-21 06:06:00,3,21,6,6,3,21,4,5,3,21,4,35


In [ ]:
dm.drop(['datum_ukupno'], axis=1)

,channel,title,kategorija,repriza,broj_gledatelja,vrijeme,mjesec,dan,sat,minuta,mjesec_start,dan_start,sat_start,minuta_start,mjesec_stop,dan_stop,sat_stop,minuta_stop
8,1,988,6,0,397,00:00:00,11,13,0,0,11,13,0,0,11,13,0,10
7,1,988,6,0,405,00:01:00,11,13,0,1,11,13,0,0,11,13,0,10
5,1,988,6,0,395,00:02:00,11,13,0,2,11,13,0,0,11,13,0,10
6,1,988,6,0,368,00:03:00,11,13,0,3,11,13,0,0,11,13,0,10
9,1,988,6,0,356,00:04:00,11,13,0,4,11,13,0,0,11,13,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155467,1,146,1,1,269,06:03:00,3,21,6,3,3,21,4,5,3,21,4,35
155468,1,146,1,1,256,06:04:00,3,21,6,4,3,21,4,5,3,21,4,35
155469,1,146,1,1,255,06:05:00,3,21,6,5,3,21,4,5,3,21,4,35
155470,1,146,1,1,246,06:06:00,3,21,6,6,3,21,4,5,3,21,4,35


In [ ]:
dm = dm[['channel', 'title', 'kategorija', 'repriza',
          'mjesec', 'dan', 'sat', 'minuta',
          'mjesec_start', 'dan_start', 'sat_start', 'minuta_start',
          'mjesec_stop', 'dan_stop', 'sat_stop', 'minuta_stop',
          'broj_gledatelja'
           ]]

In [ ]:
dm.head()

,channel,title,kategorija,repriza,mjesec,dan,sat,minuta,mjesec_start,dan_start,sat_start,minuta_start,mjesec_stop,dan_stop,sat_stop,minuta_stop,broj_gledatelja
8,1,988,6,0,11,13,0,0,11,13,0,0,11,13,0,10,397
7,1,988,6,0,11,13,0,1,11,13,0,0,11,13,0,10,405
5,1,988,6,0,11,13,0,2,11,13,0,0,11,13,0,10,395
6,1,988,6,0,11,13,0,3,11,13,0,0,11,13,0,10,368
9,1,988,6,0,11,13,0,4,11,13,0,0,11,13,0,10,356


In [ ]:
dm2_list = dm.values.tolist()

In [ ]:
dm2_list

[[1, 988, 6, 0, 11, 13, 0, 0, 11, 13, 0, 0, 11, 13, 0, 10, 397],
 [1, 988, 6, 0, 11, 13, 0, 1, 11, 13, 0, 0, 11, 13, 0, 10, 405],
 [1, 988, 6, 0, 11, 13, 0, 2, 11, 13, 0, 0, 11, 13, 0, 10, 395],
 [1, 988, 6, 0, 11, 13, 0, 3, 11, 13, 0, 0, 11, 13, 0, 10, 368],
 [1, 988, 6, 0, 11, 13, 0, 4, 11, 13, 0, 0, 11, 13, 0, 10, 356],
 [1, 988, 6, 0, 11, 13, 0, 5, 11, 13, 0, 0, 11, 13, 0, 10, 358],
 [1, 988, 6, 0, 11, 13, 0, 6, 11, 13, 0, 0, 11, 13, 0, 10, 362],
 [1, 988, 6, 0, 11, 13, 0, 7, 11, 13, 0, 0, 11, 13, 0, 10, 375],
 [1, 988, 6, 0, 11, 13, 0, 8, 11, 13, 0, 0, 11, 13, 0, 10, 358],
 [1, 988, 6, 0, 11, 13, 0, 9, 11, 13, 0, 0, 11, 13, 0, 10, 341],
 [1, 775, 6, 1, 11, 13, 0, 10, 11, 13, 0, 10, 11, 13, 0, 15, 313],
 [1, 775, 6, 1, 11, 13, 0, 11, 11, 13, 0, 10, 11, 13, 0, 15, 300],
 [1, 775, 6, 1, 11, 13, 0, 12, 11, 13, 0, 10, 11, 13, 0, 15, 314],
 [1, 775, 6, 1, 11, 13, 0, 13, 11, 13, 0, 10, 11, 13, 0, 15, 300],
 [1, 775, 6, 1, 11, 13, 0, 14, 11, 13, 0, 10, 11, 13, 0, 15, 320],
 [1, 994, 6, 1,

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [ ]:
def split_sequence(sequence):
  X = []
  y = []
  for e in sequence:
    seq_x = e[0:16]
    seq_y = e[16]
    X.append(seq_x)
    y.append(seq_y)
    #end_ix = end_ix + 2
  return np.array(X), np.array(y)

In [ ]:
dm100 = dm2_list

In [ ]:
dm100[1500]

[1, 989, 6, 0, 11, 14, 1, 0, 11, 14, 1, 0, 11, 14, 1, 5, 412]

In [ ]:
dm100[3361]

[1, 988, 6, 0, 11, 15, 8, 1, 11, 15, 8, 0, 11, 15, 8, 5, 1572]

In [ ]:
dm100[1920][6:8]

[8, 0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dm2_list = scaler.fit_transform(dm2_list)


#from sklearn.preprocessing import StandardScaler
#trans = StandardScaler()
#dm2_list = trans.fit_transform(dm2_list)


#from sklearn.preprocessing import StandardScaler
#trans = StandardScaler()
#dm2_list = trans.fit_transform(dm2_list)

In [ ]:
#from numpy import asarray
#from sklearn.preprocessing import StandardScaler
#data = asarray([[0], [9], [0], [0], [11], [13], [0], [24], [11], [12], [22], [5], [11], [13], [0], [25]])
#scaler = StandardScaler()
#dm2_list = scaler.fit_transform(dm2_list)

In [ ]:
X, y = split_sequence(dm2_list)

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
dm2_list[1500]

array([0.        , 0.93922127, 0.71428571, 0.        , 0.90909091,
       0.43333333, 0.04347826, 0.        , 0.90909091, 0.43333333,
       0.04347826, 0.        , 0.90909091, 0.43333333, 0.04347826,
       0.08474576, 0.01909585])

DEFINIRANJE LSTM MREZE

In [ ]:
from keras.constraints import nonneg
model = Sequential()
# activation='softmax',
model.add(LSTM(50, activation='tanh', input_shape=(16,1), return_sequences=True))
#model.add(LSTM(128, return_sequences=True))
model.add(LSTM(50))
#model.add(LSTM(128))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='rmsprop', loss='mean_squared_logarithmic_error')

In [ ]:
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
4860/4860 [==============================] - 65s 13ms/step - loss: 0.0014
Epoch 2/20
4860/4860 [==============================] - 62s 13ms/step - loss: 8.4789e-04
Epoch 3/20
4860/4860 [==============================] - 65s 13ms/step - loss: 7.8403e-04
Epoch 4/20
4860/4860 [==============================] - 64s 13ms/step - loss: 7.5121e-04
Epoch 5/20
4860/4860 [==============================] - 64s 13ms/step - loss: 6.9378e-04
Epoch 6/20
4860/4860 [==============================] - 64s 13ms/step - loss: 6.6035e-04
Epoch 7/20
4860/4860 [==============================] - 63s 13ms/step - loss: 6.0313e-04
Epoch 8/20
4860/4860 [==============================] - 63s 13ms/step - loss: 6.0599e-04
Epoch 9/20
4860/4860 [==============================] - 63s 13ms/step - loss: 5.5436e-04
Epoch 10/20
4860/4860 [==============================] - 64s 13ms/step - loss: 5.3519e-04
Epoch 11/20
4860/4860 [==============================] - 64s 13ms/step - loss: 5.1052e-04
Epoch 12/20
4860/4860 [

PREDVIDANJE

In [ ]:
proba = [[1, 767, 1, 0, 3, 21, 21, 19, 3, 21, 21, 12, 3, 21, 22, 0, 1367]]
proba = scaler.transform(proba)
print(proba)

[[0.         0.72839506 0.         0.         0.18181818 0.66666667
  0.91304348 0.3220339  0.18181818 0.66666667 0.91304348 0.20338983
  0.18181818 0.66666667 0.95652174 0.         0.06346699]]


In [ ]:
x_input = np.array([0.,         0.72839506, 0.,         0.,         0.18181818, 0.66666667,
  0.91304348, 0.3220339,  0.18181818, 0.66666667, 0.91304348, 0.20338983,
  0.18181818, 0.66666667, 0.95652174, 0. ])

In [ ]:
x_input = x_input.reshape(1, 16, 1)

In [ ]:
x_input

array([[[0.        ],
        [0.5042735 ],
        [0.28571429],
        [0.        ],
        [0.18181818],
        [0.66666667],
        [0.47826087],
        [0.40677966],
        [0.18181818],
        [0.66666667],
        [0.47826087],
        [0.16949153],
        [0.18181818],
        [0.66666667],
        [0.47826087],
        [0.59322034]]])

In [ ]:
predictions = model.predict(x_input)

In [ ]:
print(predictions)

[[0.04306337]]


In [ ]:
p = np.array([0.,         0.9382716,  0.71428571, 0.,         0.18181818, 0.7,
  0.43478261, 0.,         0.18181818, 0.7,        0.43478261, 0.,
  0.18181818, 0.7,        0.43478261, 0.16949153 , predictions])
p = scaler.inverse_transform([p])
print(p)

[[  1.         987.9999948    5.99999997   0.           2.99999998
   22.          10.00000003   0.           2.99999998  22.
   10.00000003   0.           2.99999998  22.          10.00000003
   10.00000027 927.8529636 ]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
